In [11]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [12]:
def get_odds_data():
    sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': api_key,
        'sport': 'basketball_ncaab',
        'region': 'us',
        'oddsFormat': 'american'
    })


    sports_json = json.loads(sports_response.text)
    date = dt.datetime.today().date().strftime('%Y-%m-%d')
    with open(str(date)+'.json', 'w') as outfile:
        json.dump(sports_json, outfile)
    

In [13]:
get_odds_data()

In [14]:
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json') as f:
    sports_json = json.load(f)

In [15]:
def getTeamData(name):
    name = name.replace(" ", "%20")
    baseUrl="https://api.fastbreakstats.com/teams/"
    params="?q=current"
    url=baseUrl+name+params

    r = requests.get(url = url).json()[0]
    bpival = r['BPIval']
    kpval = r['KenPomVal']
    srsval = r['SRS']
    return kpval, bpival, srsval

In [16]:
def get_needed_odds1(conf):
    payout = round(((.1+(1-conf))/conf),4)
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [17]:
def get_ev1(odds,conf):
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
    ev = (conf * payout) - (1-conf)
    return ev

In [18]:
def print_games():
    teams = pd.read_csv('teams_table.csv')
    loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
    for game in sports_json['data']:
        #Loop through and get odds
        odds = ['n/a','n/a']
        for site in game['sites']:
            if site['site_key'] == 'bovada':
                odds = site['odds']['h2h']
                sitex = "bovada"
                break
            if site['site_key'] == 'draftkings':
                odds = site['odds']['h2h']
                sitex = "DK"
            if site['site_key'] == 'williamhill_us':
                odds = site['odds']['h2h']
                sitex = "WH"
            if site['site_key'] == 'fanduel':
                odds = site['odds']['h2h']
                sitex = "FD"

        if odds == ['n/a','n/a']:
            print("There are no odds for {0}".format(game['teams']))
            print()
            continue

        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
            home_odds = odds[0]
            away_odds = odds[1]
        else:
            away = game['teams'][0]
            home_odds = odds[1]
            away_odds = odds[0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
            awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
        except Exception as e:
            print(e)
            print(game['teams'])
            print()
            continue

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
    #     print(res[0])
        print("{0} @ {1}".format(away,home))
        away_needed_odds = get_needed_odds1(res[0][0])    
        home_needed_odds = get_needed_odds1(res[0][1])
        away_ev = get_ev1(away_odds,res[0][0])
        home_ev = get_ev1(home_odds,res[0][1])
        if away_odds > away_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(away,away_needed_odds,away_odds,away_ev))
#             print()
        if home_odds > home_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(home,home_needed_odds,home_odds,home_ev))
        print()

In [19]:
print_games()

c:\users\blake\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Maine Black Bears @ Boston College Eagles

SE Missouri St Redhawks @ Indiana St Sycamores

Alcorn St Braves @ Liberty Flames

Jackson St Tigers @ North Alabama Lions

Ohio Bobcats @ Akron Zips

Charlotte 49ers @ George Washington Colonials
Charlotte 49ers @ George Washington Colonials on bovada
Charlotte 49ers needs -131.0 and has -120 odds with an Expected Value of 0.1447362530030326

Northern Illinois Huskies @ Toledo Rockets

Ball State Cardinals @ Western Michigan Broncos

William & Mary Tribe @ Virginia Cavaliers

Hofstra Pride @ Richmond Spiders

Mercer Bears @ Kennesaw St Owls

Northern Arizona Lumberjacks @ Denver Pioneers
Northern Arizona Lumberjacks @ Denver Pioneers on DK
Denver Pioneers needs -176.0 and has -175 odds with an Expected Value of 0.10117676277524573

Appalachian St Mountaineers @ Auburn Tigers

Montana St Bobcats @ Portland Pilots

Chattanooga Mocs @ Tennessee St Tigers
Chattanooga Mocs @ Tennessee St Tigers on bovada
Chattanooga Mocs needs -280.0 and has -190 

In [41]:
teams = pd.read_csv('teams_table.csv')
awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == 'Tenn-Martin Skyhawks']['kenpom_name'].values[0])

In [19]:
def get_bet_games():
    teams = pd.read_csv('teams_table.csv')
    loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
    games_to_bet = []
    for game in sports_json['data']:
        #Loop through and get odds
        odds = ['n/a','n/a']
        for site in game['sites']:
            if site['site_key'] == 'bovada':
                odds = site['odds']['h2h']
                sitex = "bovada"
                break
            if site['site_key'] == 'draftkings':
                odds = site['odds']['h2h']
                sitex = "DK"
            if site['site_key'] == 'williamhill_us':
                odds = site['odds']['h2h']
                sitex = "WH"
            if site['site_key'] == 'fanduel':
                odds = site['odds']['h2h']
                sitex = "FD"

        if odds == ['n/a','n/a']:
    #         print("There are no odds for {0}".format(game['teams']))
            continue

        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
            home_odds = odds[0]
            away_odds = odds[1]
        else:
            away = game['teams'][0]
            home_odds = odds[1]
            away_odds = odds[0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
            awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
        except Exception as e:
            print(e)
            print(game['teams'])
            pass

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
        away_needed_odds = get_needed_odds1(res[0][0])    
        home_needed_odds = get_needed_odds1(res[0][1])
        away_ev = get_ev1(away_odds,res[0][0])
        home_ev = get_ev1(home_odds,res[0][1])
        if away_odds > away_needed_odds:
            game_to_bet = {}
            game_to_bet['game'] = "{0} @ {1}".format(away,home)
            game_to_bet['starttime'] = dt.datetime.fromtimestamp(game['commence_time']).strftime('%Y-%m-%d %H:%M:%S CST')
            game_to_bet['betsite'] = sitex
            game_to_bet['team'] = away
            game_to_bet['odds_found'] = away_odds
            game_to_bet['odds_needed'] = away_needed_odds
            game_to_bet['ev'] = away_ev
            games_to_bet.append(game_to_bet)
        if home_odds > home_needed_odds:
            game_to_bet = {}
            game_to_bet['game'] = "{0} @ {1}".format(away,home)
            game_to_bet['starttime'] = dt.datetime.fromtimestamp(game['commence_time']).strftime('%Y-%m-%d %H:%M:%S CST')
            game_to_bet['betsite'] = sitex
            game_to_bet['team'] = home
            game_to_bet['odds_found'] = home_odds
            game_to_bet['odds_needed'] = home_needed_odds
            game_to_bet['ev'] = home_ev
            games_to_bet.append(game_to_bet)
    return games_to_bet

In [20]:
today = get_bet_games()

index 0 is out of bounds for axis 0 with size 0
['Monmouth Hawks', "Saint Peter's Peacocks"]
index 0 is out of bounds for axis 0 with size 0
['BYU Cougars', 'San Diego St Aztecs']
index 0 is out of bounds for axis 0 with size 0
['Charleston Cougars', 'Western Carolina Catamounts']
index 0 is out of bounds for axis 0 with size 0
['Northwestern St Demons', 'Tulsa Golden Hurricane']
index 0 is out of bounds for axis 0 with size 0
['SE Missouri St Redhawks', 'Tenn-Martin Skyhawks']
index 0 is out of bounds for axis 0 with size 0
['San José St Spartans', 'Santa Clara Broncos']
index 0 is out of bounds for axis 0 with size 0
['N Colorado Bears', 'Utah State Aggies']
index 0 is out of bounds for axis 0 with size 0
['Portland St Vikings', 'Weber State Wildcats']
index 0 is out of bounds for axis 0 with size 0
['Ball State Cardinals', 'Indiana St Sycamores']
index 0 is out of bounds for axis 0 with size 0
['Detroit Mercy Titans', 'Wright St Raiders']


In [21]:
print(today)

[{'game': 'Central Michigan Chippewas @ Western Michigan Broncos', 'starttime': '2020-12-18 13:00:00 CST', 'betsite': 'bovada', 'team': 'Western Michigan Broncos', 'odds_found': -115, 'odds_needed': -118.0, 'ev': 0.11238818502083503}, {'game': 'UNC Wilmington Seahawks @ Norfolk St Spartans', 'starttime': '2020-12-18 15:00:00 CST', 'betsite': 'bovada', 'team': 'Norfolk St Spartans', 'odds_found': 100, 'odds_needed': -160.0, 'ev': 0.3542868277621136}, {'game': "Saint Peter's Peacocks @ Monmouth Hawks", 'starttime': '2020-12-18 16:00:00 CST', 'betsite': 'bovada', 'team': 'Monmouth Hawks', 'odds_found': 100, 'odds_needed': -221.0, 'ev': 0.5156244689382481}, {'game': 'Hampton Pirates @ Howard Bison', 'starttime': '2020-12-18 16:00:00 CST', 'betsite': 'bovada', 'team': 'Howard Bison', 'odds_found': 110, 'odds_needed': -130.0, 'ev': 0.3069218188254488}, {'game': 'BYU Cougars @ San Diego St Aztecs', 'starttime': '2020-12-18 16:00:00 CST', 'betsite': 'DK', 'team': 'San Diego St Aztecs', 'odds_f